In [1]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
class Encoder(nn.Module):
    def __init__(self, batch_size:int, model_dim:int=512, num_heads:int=8):
        super().__init__()
        self.num_stacks = 6
        self.attention = nn.MultiheadAttention(embed_dim=model_dim, num_heads=num_heads)  ### IMPLEMENT FROM SCRATCH
        self.ffnn = nn.Sequential(
            nn.Linear(model_dim, 2048),
            nn.ReLU(),
            nn.Linear(2048, model_dim)
        )
        self.layernorm = nn.LayerNorm(normalized_shape=model_dim)

        self.batch_size = batch_size

        self.queryM = nn.Parameter(torch.randn(model_dim, num_heads, model_dim//num_heads))  ### First find out exact model input from embedding
        self.keyM = nn.Parameter(torch.randn(model_dim, num_heads, model_dim//num_heads))    ### Then find out how to make this tensor such that it multiplies the input embedding and has the output (seq_len, batch, model_dim)
        self.valueM = nn.Parameter(torch.randn(model_dim, num_heads,model_dim//num_heads))

    def forward(self, inputs):
        return self._stack(inputs, self.num_stacks)

    def _stack(self, input, num_stacks):
        if num_stacks < 1:
            return input
        
        query = input @ self.queryM
        key = input @ self.keyM
        value = input @ self.valueM

        layer = self.layernorm(input + self.attention.forward(query=query, key=key, value=value))
        
        output = self.layernorm(layer + self.ffnn(layer))

        return self._stack(output, num_stacks-1)    